# Use MongoDBQueryEngine to query Markdown files 

This notebook demonstrates the use of the `MongoDBQueryEngine` for retrieval-augmented question answering over documents. It shows how to set up the engine with Docling parsed Markdown files, and execute natural language queries against the indexed data. 

The `MongoDBQueryEngine` integrates cloud MongoDB Atlas vector storage with LlamaIndex for efficient document retrieval.

In [ ]:
%pip install llama-index==0.12.16
%pip install llama-index-vector-stores-mongodb==0.6.0
%pip install llama-index-embeddings-huggingface==0.5.2

Before calling the `MongoDBQueryEngine`, we will create and import our own embedding function. You can replace this with any embedding function built-in from Langchain or Llama-Index or your custom build

In [ ]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

To use this notebook, you will need to have a MongoDB Atlas account, please refer [MongoDB Atlas Signup](https://www.mongodb.com/cloud/atlas/register?msockid=1dc5ecf40bf26a7f042df9790ae06b20) for more info. When you create account or already have the account, MongoDB Atlas will provide and support you free version as well. So you can use it to run this tutorial

Some info that you need to get include:
- MongoDB Connection String (URI)

In [ ]:
from autogen.agentchat.contrib.rag.mongodb_query_engine import MongoDBQueryEngine

query_engine = MongoDBQueryEngine(connection_string="", database_name="vector_db", collection_name="test_collection")

Here we can see the default collection name in the vector store, this is where all documents will be ingested. When creating the `MongoDBQueryEngine` you can specify a `collection_name` and `database_name` to ingest into.

In [ ]:
print(query_engine.get_collection_name())

Let's ingest a document and query it.  
 ```init_db``` will overwrite the existing collection with the same name.

In [ ]:
input_dir = "/root/ag2/test/agents/experimental/document_agent/pdf_parsed/"  # Update to match your input directory

In [ ]:
input_docs = [input_dir + "nvidia_10k_2024.md"]  # Update to match your input documents
query_engine.init_db(new_doc_paths_or_urls=input_docs)

If the given collection already has the document you need, you can use ```connect_db``` to avoid overwriting the existing collection.

In [ ]:
# query_engine.connect_db()

In [ ]:
question = "How much money did Nvidia spend in research and development"
answer = query_engine.query(question)
print(answer)

Great, we got the data we needed. Now, let's add another document.

In [ ]:
new_docs = [input_dir + "Toast_financial_report.md"]
query_engine.add_docs(new_doc_paths_or_urls=new_docs)

And query again from the same database but this time for another corporate entity.

In [ ]:
question = "What is the trading symbol for Toast"
answer = query_engine.query(question)
print(answer)